# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - Queue Management

**Subtopic** - Gracefully shutting down a queue processing system by draining the queue

**Use Case** - Implement a simple queue processing system using Python's built-in queue module. The system should be able to add tasks to the queue, process tasks from the queue, and gracefully shut down by draining the queue before termination. The queue processing system should be designed in such a way that it does not lose any tasks during shutdown. The shutdown process should involve stopping the addition of new tasks to the queue, processing all remaining tasks in the queue, and then terminating the system. The system should be able to handle any exceptions during task processing, ensuring that it does not crash unexpectedly.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt
Problem Statement:
- Implement a simple queue processing system using Python's built-in queue module.
- The system should be able to add tasks to the queue, process tasks from the queue, and gracefully shut down by draining the queue before termination.
- The queue processing system should be designed in such a way that it does not lose any tasks during shutdown.
- The shutdown process should involve stopping the addition of new tasks to the queue, processing all remaining tasks in the queue, and then terminating the system.
- The system should be able to handle any exceptions during task processing, ensuring that it does not crash unexpectedly.

Input Format:
- Tasks are added programmatically via a function or method call.
- Each task is represented as a tuple: (callable_func, args, kwargs).
- callable_func: A reference to the function to execute.
- args: A tuple of positional arguments for the function.
- kwargs: A dictionary of keyword arguments for the function.
- The system runs continuously until a shutdown is triggered.

Input Constraints:
- Each task must be a 3-tuple: (callable_func, args, kwargs).
- callable_func must be a callable object (e.g., function or method).
- args must be a tuple of positional arguments (can be empty).
- kwargs must be a dictionary of keyword arguments (can be empty).
- All elements in args and kwargs must be serializable and safe for function execution.

Output Format:
- The system performs task execution internally.
- Successful task execution must be printed.
- During shutdown, the system ensures all tasks are processed and no data is lost.
- Any exceptions during task execution are handled and printed without terminating the system.

Class Definition and Function Signature:
```python
class QueueProcessor:
    def __init__(self) -> None:
        """Initialize the task queue, worker thread, and shutdown flag."""
        pass

    def add_task(
        self,
        callable_func: Callable,
        args: Tuple[Any, ...],
        kwargs: Dict[str, Any]
    ) -> None:
        """Add a new task to the queue if not shutting down."""
        pass

    def _worker(self) -> None:
        """Continuously process tasks from the queue until shutdown is complete."""
        pass

    def shutdown(self) -> None:
        """Gracefully shut down the system after draining the queue."""
        pass

    def start(self) -> None:
        """Start the worker thread to begin processing tasks."""
        pass


def start_queue_processor() -> None:
    """Initialize and run the queue processor until shutdown is triggered."""
    pass
```


# Requirements
Explicit Requirements:
- Use Python's built-in queue module for task management.
- Each task must be a tuple: (callable_func, args, kwargs).
- The system must support adding tasks, processing tasks, and graceful shutdown.
- During shutdown, new task additions must be rejected.
- All remaining tasks in the queue must be processed before termination.
- Exceptions during task processing must be handled and printed.
- Successful task execution must be printed.
- The system must not crash or lose tasks during execution or shutdown.

Implicit Requirements:
- The system should run in a separate thread for background processing.
- Task processing should be asynchronous from task submission.
- The shutdown process should be thread-safe and idempotent.
- The system should support multiple tasks being added before and during processing.
- The queue size can be unbounded unless explicitly modified later.
- The system must maintain internal state (e.g., is_shutting_down flag) to control behavior.
- Logging or print statements may be used for debugging and tracking task outcomes.

Solution Expectations:
- Define a class (e.g., QueueProcessor) to encapsulate queue operations, threading, and shutdown logic.
- Use Python's queue.Queue for managing task storage and retrieval.
- Implement a background worker thread that continuously processes tasks from the queue.
- Provide a method to add tasks only when the system is not shutting down.
- Ensure tasks are executed safely with exception handling to prevent crashes.
- Implement a shutdown method that:
  - Stops acceptance of new tasks,
  - Waits until the queue is empty,
  - Joins the worker thread to ensure clean termination.
- Print output for each successfully completed task.
- Print error details when exceptions occur during task execution.
- Ensure the system can scale to handle multiple tasks being added and processed concurrently.
- Avoid data loss during both normal operation and shutdown.
- Perform input validation on all inputs.

Edge Cases and Behavior:
- Adding a task after shutdown has been initiated:
  - The task must be rejected and not added to the queue.
- Submitting an invalid task (not a tuple or incorrect structure):
  - The system should raise a clear exception or ignore the task safely.
- Exceptions raised during task execution:
  - The exception must be caught and printed; processing must continue for other tasks.
- No tasks are added before shutdown:
  - The system should shut down cleanly without processing anything.
- A long-running task is being processed during shutdown:
  - The system must wait for the task to complete before termination.
- Multiple tasks added in rapid succession:
  - The system should process them in order without crashing or losing any.
- Simultaneous calls to shutdown():
  - Shutdown must be thread-safe and idempotent (should have the same effect if called multiple times).
- Callable raises a non-standard exception:
  - The exception must be handled gracefully without terminating the worker.

Solution Constraints:
- Only standard Python libraries are allowed (no third-party packages).
- Must use Python's built-in queue.Queue for task management.
- Task processing must occur on a separate worker thread.
- The system must handle up to thousands of tasks without failure or data loss.
- The shutdown process must ensure all tasks in the queue are completed before exit.
- No new tasks may be accepted once shutdown begins.
- Exceptions during task execution must not terminate the worker thread.
- Task submission and shutdown logic must be thread-safe.
- The system must print a message for each successful task execution.
- The system must print error messages for failed task executions without exiting.

In [ ]:
# code
"""QueueProcessor system to manage tasks using queue module."""

import queue
import threading
import time
from typing import Callable, Any, Tuple, Dict


class QueueProcessor:
    """
    A simple queue processing system using Python's built-in queue module.

    The system can add tasks to the queue, process tasks from the queue,
    and gracefully shut down by draining the queue before termination,
    ensuring no tasks are lost.
    """

    def __init__(self) -> None:
        """Initialize the task queue, worker thread, and shutdown flag."""
        self._task_queue: queue.Queue = queue.Queue()
        self._shutdown_flag = threading.Event()
        self._worker_thread: threading.Thread | None = None
        self._lock = threading.Lock()

    def add_task(
        self,
        callable_func: Callable,
        args: Tuple[Any, ...],
        kwargs: Dict[str, Any]
    ) -> None:
        """
        Add a new task to the queue if the system is not shutting down.

        Args:
            callable_func: The function to execute.
            args: A tuple of positional arguments for the function.
            kwargs: A dictionary of keyword arguments for the function.

        Raises:
            ValueError: If the task is not a 3-tuple or has incorrect types.
            RuntimeError: If adding a task while the system is shutting down.
        """
        with self._lock:
            if self._shutdown_flag.is_set():
                raise RuntimeError("Cannot add tasks:"
                                   " system is shutting down.")

            if not isinstance(callable_func, Callable):
                raise ValueError("callable_func must be a callable object.")
            if not isinstance(args, tuple):
                raise ValueError("args must be a tuple.")
            if not isinstance(kwargs, dict):
                raise ValueError("kwargs must be a dictionary.")

            self._task_queue.put((callable_func, args, kwargs))
            print(f"Task added: {callable_func.__name__}")

    def _worker(self) -> None:
        """Continuously process tasks from the queue."""
        while True:
            try:
                task = self._task_queue.get(timeout=0.1)
                callable_func, args, kwargs = task
                try:
                    result = callable_func(*args, **kwargs)
                    print(
                        f"Successfully executed task:"
                        f" {callable_func.__name__}, "
                        f"Result: {result}"
                    )
                except Exception as e:
                    print(f"Error processing task"
                          f" {callable_func.__name__}: {e}")
                finally:
                    self._task_queue.task_done()
            except queue.Empty:
                if self._shutdown_flag.is_set():
                    break
            except Exception as e:
                print(f"Unexpected error in worker thread: {e}")
                if self._shutdown_flag.is_set():
                    break

        print("Worker thread finished processing remaining tasks.")

    def start(self) -> None:
        """Start the worker thread to begin processing tasks."""
        with self._lock:
            if (self._worker_thread is not None
                    and self._worker_thread.is_alive()):
                print("Queue processor is already running.")
                return
            self._shutdown_flag.clear()
            self._worker_thread = threading.Thread(
                target=self._worker,
                name="QueueProcessorWorker"
            )
            self._worker_thread.start()
            print("Queue processor started.")

    def shutdown(self) -> None:
        """
        Shut down the system gracefully after draining the queue.

        This method is thread-safe and idempotent.
        """
        with self._lock:
            if self._shutdown_flag.is_set():
                print("Shutdown already initiated or completed.")
                return

            print("Initiating graceful shutdown...")
            self._shutdown_flag.set()

        print("Waiting for all pending tasks to complete...")
        self._task_queue.join()

        if self._worker_thread and self._worker_thread.is_alive():
            self._worker_thread.join()

        print("Queue processor shut down successfully. All tasks processed.")


def start_queue_processor() -> None:
    """
    Initialize and run the queue processor until shutdown is triggered.

    Demonstrate adding tasks and handling shutdown.
    """
    processor = QueueProcessor()
    processor.start()

    def sample_task(task_id: int, duration: float = 0.1):
        """Return success message after sleeping briefly."""
        time.sleep(duration)
        return f"Task {task_id} completed."

    def failing_task(task_id: int):
        """Raise simulated error for the task."""
        raise ValueError(f"Simulated error for Task {task_id}")

    print("\nAdding initial tasks...")
    for i in range(5):
        processor.add_task(sample_task, (i,), {})
    processor.add_task(failing_task, (99,), {})  # A task that will fail

    time.sleep(1)

    print("\nAdding more tasks during operation...")
    for i in range(5, 10):
        processor.add_task(sample_task, (i,), {'duration': 0.05})

    print("\nTriggering shutdown in 2 seconds...")
    time.sleep(2)
    processor.shutdown()

    try:
        processor.add_task(sample_task, (100,), {})
    except RuntimeError as e:
        print(f"\nCaught expected error: {e}")

    print("\nAttempting to call shutdown again (should be idempotent)...")
    processor.shutdown()

    print("\nQueue processor demonstration finished.")


if __name__ == "__main__":
    start_queue_processor()


In [ ]:
# tests

"""
Test cases for the QueueProcessor system.

This module contains comprehensive test cases for testing the QueueProcessor
class and its functionality including task addition, processing, exception
handling, and graceful shutdown.
"""

import unittest
import threading
import time
from unittest.mock import patch
from io import StringIO
from main import QueueProcessor, start_queue_processor


class TestQueueProcessor(unittest.TestCase):
    """Test cases for QueueProcessor class."""

    def setUp(self):
        """Set up test fixtures before each test method."""
        self.processor = QueueProcessor()
        self.test_results = []
        self.test_lock = threading.Lock()

    def tearDown(self):
        """Clean up after each test method."""
        if hasattr(self.processor, '_worker_thread'):
            if self.processor._worker_thread.is_alive():
                self.processor.shutdown()
        self.test_results.clear()

    def sample_task(self, value: int, multiplier: int = 1) -> int:
        """
        Sample task function for testing.

        Args:
            value: Integer value to process
            multiplier: Multiplier for the value

        Returns:
            Result of value * multiplier
        """
        result = value * multiplier
        with self.test_lock:
            self.test_results.append(result)
        return result

    def slow_task(self, duration: float = 0.1) -> str:
        """
        Slow task that takes time to complete.

        Args:
            duration: Time to sleep in seconds

        Returns:
            Completion message
        """
        time.sleep(duration)
        with self.test_lock:
            self.test_results.append(f"slow_task_completed_{duration}")
        return f"completed after {duration}s"

    def failing_task(self, error_msg: str = "Test error") -> None:
        """
        Task that always raises an exception.

        Args:
            error_msg: Error message to raise

        Raises:
            ValueError: Always raises with given message
        """
        raise ValueError(error_msg)

    def test_add_single_task_and_process(self):
        """Test adding a single task and processing it."""
        self.processor.start()

        self.processor.add_task(self.sample_task, (5,), {'multiplier': 2})

        # Wait for task to be processed
        time.sleep(0.1)

        with self.test_lock:
            self.assertEqual(len(self.test_results), 1)
            self.assertEqual(self.test_results[0], 10)

        self.processor.shutdown()

    def test_add_multiple_tasks_sequentially(self):
        """Test adding multiple tasks sequentially."""
        self.processor.start()

        for i in range(5):
            self.processor.add_task(self.sample_task, (i,), {'multiplier': 2})

        # Wait for all tasks to be processed
        time.sleep(0.2)

        with self.test_lock:
            self.assertEqual(len(self.test_results), 5)
            expected_results = [i * 2 for i in range(5)]
            self.assertEqual(sorted(self.test_results), sorted(expected_results))

        self.processor.shutdown()

    def test_add_tasks_with_empty_args_and_kwargs(self):
        """Test adding tasks with empty args and kwargs."""
        self.processor.start()

        def simple_task():
            with self.test_lock:
                self.test_results.append("simple_task_completed")
            return "done"

        self.processor.add_task(simple_task, (), {})

        time.sleep(0.1)

        with self.test_lock:
            self.assertEqual(len(self.test_results), 1)
            self.assertEqual(self.test_results[0], "simple_task_completed")

        self.processor.shutdown()

    def test_task_execution_with_exception_handling(self):
        """Test that exceptions in tasks are handled gracefully."""
        self.processor.start()

        # Add a failing task
        self.processor.add_task(self.failing_task, ("Custom error",), {})

        # Add a successful task after the failing one
        self.processor.add_task(self.sample_task, (3,), {'multiplier': 4})

        time.sleep(0.2)

        # The successful task should still complete
        with self.test_lock:
            self.assertEqual(len(self.test_results), 1)
            self.assertEqual(self.test_results[0], 12)

        self.processor.shutdown()

    @patch('sys.stdout', new_callable=StringIO)
    def test_exception_handling_output(self, mock_stdout):
        """Test that exceptions are printed without crashing."""
        self.processor.start()

        self.processor.add_task(self.failing_task, ("Test exception",), {})

        time.sleep(0.1)
        self.processor.shutdown()

        output = mock_stdout.getvalue()
        self.assertIn("Error processing task", output)
        self.assertIn("Test exception", output)

    def test_graceful_shutdown_with_pending_tasks(self):
        """Test graceful shutdown with tasks still in queue."""
        self.processor.start()

        # Add multiple tasks
        for i in range(3):
            self.processor.add_task(self.sample_task, (i,), {'multiplier': 1})

        # Shutdown immediately
        self.processor.shutdown()

        # All tasks should be processed
        with self.test_lock:
            self.assertEqual(len(self.test_results), 3)
            self.assertEqual(sorted(self.test_results), [0, 1, 2])

    def test_shutdown_with_long_running_task(self):
        """Test shutdown waits for long-running tasks to complete."""
        self.processor.start()

        # Add a slow task
        self.processor.add_task(self.slow_task, (0.2,), {})

        # Start shutdown
        start_time = time.time()
        self.processor.shutdown()
        end_time = time.time()

        # Should have waited for the task to complete
        self.assertGreater(end_time - start_time, 0.2)

        with self.test_lock:
            self.assertEqual(len(self.test_results), 1)
            self.assertEqual(self.test_results[0], "slow_task_completed_0.2")

    def test_reject_tasks_after_shutdown_initiated(self):
        """Test that tasks are rejected after shutdown is initiated."""
        self.processor.start()

        # Start shutdown in a separate thread
        shutdown_thread = threading.Thread(target=self.processor.shutdown)
        shutdown_thread.start()

        # Try to add a task after shutdown started
        time.sleep(0.05)  # Small delay to ensure shutdown started

        # This should not add the task
        with self.assertRaises(RuntimeError):
            self.processor.add_task(self.sample_task, (10,), {})

    def test_multiple_shutdown_calls_idempotent(self):
        """Test that multiple shutdown calls are handled safely."""
        self.processor.start()

        # First shutdown
        self.processor.shutdown()

        # Second shutdown should not cause issues
        self.processor.shutdown()

    def test_shutdown_with_empty_queue(self):
        """Test shutdown with no tasks in queue."""
        self.processor.start()

        # Shutdown immediately without adding tasks
        self.processor.shutdown()

        # Should complete without errors
        with self.test_lock:
            self.assertEqual(len(self.test_results), 0)

    def test_concurrent_task_additions(self):
        """Test adding tasks concurrently from multiple threads."""
        self.processor.start()

        def add_tasks(start_value: int, count: int):
            for i in range(count):
                self.processor.add_task(
                    self.sample_task,
                    (start_value + i,),
                    {'multiplier': 1}
                )

        # Create multiple threads to add tasks
        threads = []
        for i in range(3):
            thread = threading.Thread(target=add_tasks, args=(i * 10, 5))
            threads.append(thread)
            thread.start()

        # Wait for all threads to complete
        for thread in threads:
            thread.join()

        # Wait for processing
        time.sleep(0.3)

        with self.test_lock:
            self.assertEqual(len(self.test_results), 15)

        self.processor.shutdown()

    def test_invalid_task_parameters(self):
        """Test handling of invalid task parameters."""
        self.processor.start()

        # Test with invalid callable
        with self.assertRaises(ValueError):
            self.processor.add_task("not_a_function", (), {})

        # Test with invalid args (not a tuple)
        with self.assertRaises(ValueError):
            self.processor.add_task(self.sample_task, "not_a_tuple", {})

        # Test with invalid kwargs (not a dict)
        with self.assertRaises(ValueError):
            self.processor.add_task(self.sample_task, (), "not_a_dict")

        self.processor.shutdown()

    def test_task_with_non_standard_exception(self):
        """Test handling of non-standard exceptions in tasks."""
        self.processor.start()

        def custom_exception_task():
            class CustomError(Exception):
                """Base Class for custom error."""
            raise CustomError("Custom exception message")

        self.processor.add_task(custom_exception_task, (), {})

        # Add a normal task after the exception
        self.processor.add_task(self.sample_task, (1,), {'multiplier': 5})

        time.sleep(0.2)

        # The normal task should still complete
        with self.test_lock:
            self.assertEqual(len(self.test_results), 1)
            self.assertEqual(self.test_results[0], 5)

        self.processor.shutdown()

    def test_high_volume_task_processing(self):
        """Test processing a high volume of tasks."""
        self.processor.start()

        task_count = 100
        for i in range(task_count):
            self.processor.add_task(self.sample_task, (i,), {'multiplier': 1})

        # Wait for all tasks to be processed
        time.sleep(1.0)

        with self.test_lock:
            self.assertEqual(len(self.test_results), task_count)
            expected_results = list(range(task_count))
            self.assertEqual(sorted(self.test_results), expected_results)

        self.processor.shutdown()

    def test_worker_thread_state_after_shutdown(self):
        """Test that worker thread terminates properly after shutdown."""
        self.processor.start()

        # Verify worker thread is alive
        self.assertTrue(self.processor._worker_thread.is_alive())

        # Shutdown
        self.processor.shutdown()

        # Worker thread should be terminated
        self.assertFalse(self.processor._worker_thread.is_alive())


class TestStartQueueProcessor(unittest.TestCase):
    """Test cases for start_queue_processor function."""

    @patch('builtins.input', return_value='quit')
    def test_start_queue_processor_basic(self, mock_input):
        """Test basic functionality of start_queue_processor."""
        # This test assumes the function exists and can be called
        # The actual implementation would need to be tested based on
        # how the function is designed to work
        try:
            start_queue_processor()
        except NameError:
            # Function not implemented yet
            self.skipTest("start_queue_processor not implemented")


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/01e48fbb-65cf-4524-a047-27c3f2fa70d4?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F82773%2Fview
#### Model code:

```python
# code generated by the model
import queue
import threading
from typing import Callable, Any, Tuple, Dict

class QueueProcessor:
    def __init__(self) -> None:
        """Initialize the task queue, worker thread, and shutdown flag."""
        self.task_queue = queue.Queue()
        self.is_shutting_down = False
        self.lock = threading.Lock()
        self.worker_thread = threading.Thread(target=self._worker)
        self.worker_thread.start()
        # <Issue>: Worker thread is started in __init__ instead of start()

    def add_task(
        self,
        callable_func: Callable,
        args: Tuple[Any, ...] = (),
        kwargs: Dict[str, Any] = {}
    ) -> None:
        """Add a new task to the queue if not shutting down."""
        if not callable(callable_func):
            print("Attempted to add a non-callable task. Ignoring.")
            return
        # <Issue>: Missing input validation for args and kwargs types (must be tuple and dict)
        with self.lock:
            if self.is_shutting_down:
                print("System is shutting down, cannot add new tasks.")
                return
            self.task_queue.put((callable_func, args, kwargs))
            print(f"Task added: {callable_func.__name__}")

    def _worker(self) -> None:
        """Continuously process tasks from the queue until shutdown is complete."""
        while True:
            task = self.task_queue.get()
            if task is None:  # Sentinel value to terminate the loop
                break
            callable_func, args, kwargs = task
            try:
                result = callable_func(*args, **kwargs)
                print(f"Task {callable_func.__name__} completed with result: {result}")
            except Exception as e:
                print(f"Exception occurred during task {callable_func.__name__}: {e}")
            finally:
                self.task_queue.task_done()

    def shutdown(self) -> None:
        """Gracefully shut down the system after draining the queue."""
        with self.lock:
            self.is_shutting_down = True
        self.task_queue.join()  # Wait until all tasks are processed
        self.task_queue.put(None)  # Enqueue sentinel to stop the worker loop
        self.worker_thread.join()  # Wait for worker thread to finish
        print("System shutdown complete.")

    def start(self) -> None:
        """Start the worker thread to begin processing tasks."""
        if not self.worker_thread.is_alive():
            self.worker_thread = threading.Thread(target=self._worker)
            self.worker_thread.start()
            # <Issue>: Worker thread was already started in __init__, so start() is redundant

def start_queue_processor() -> None:
    """Initialize and run the queue processor until shutdown is triggered."""
    processor = QueueProcessor()

    # Example of adding tasks
    processor.add_task(print, args=("Hello,",), kwargs={"end": " "})
    processor.add_task(print, args=("world!",), kwargs={"end": "\n"})

    # Simulate some operation time
    import time
    time.sleep(1)

    # Begin shutdown
    processor.shutdown()

# Example usage
if __name__ == "__main__":
    start_queue_processor()
```